In [28]:
%reload_ext autoreload
%autoreload 2

In [29]:
# Imports
import logregression as lr
import numpy as np
import numpy.ma as ma
import pandas as pd
import librosa as lb
import librosa.display

In [30]:
# Converts the CSV to a dataframe
df = pd.read_csv("speakers_all.csv")
# Creates a boolean array to determine if a file is missing or not
hasFile = np.asarray(df["file_missing?"].to_list())

In [31]:
# Gets the labels from the dataframe and returns them as US and non-US
all = np.asarray(df["native_language"])
isUS = np.isin(all, "usa")
labels = ma.masked_array(isUS, hasFile, fill_value = None)

In [32]:
# Gets audio data needed to find audio features
# currently takes about 2.5 minutes
data = []
srate = []

allFileNames = np.asarray(df["filename"])
onlyHasFile = ma.masked_array(allFileNames, hasFile, fill_value = None)

for file in onlyHasFile:
    if file != None:
        filename = "recordings/recordings/" + file + ".mp3"
        currData, currSR = lb.load(filename)
        data.append(currData)
        srate.append(currSR)

In [35]:
# Gets the audio features of each file
# currently takes about 3.5 minutes
mfccs = []

for i, j in zip(data, srate):
    mfccs.append(librosa.feature.mfcc(y=i, sr=j))

In [36]:
print(mfccs)

mfccArray = np.array([elem for elem in enumerate(mfccs)])
labelArray = np.asarray(labels)

print(mfccs)
print(labels)

lr.main(mfccArray, labelArray)

[array([[-407.85757  , -387.25946  , -377.90118  , ..., -441.32175  ,
        -444.9362   , -448.96732  ],
       [  80.291916 ,   94.30681  ,   96.39616  , ...,   59.661114 ,
          55.342346 ,   50.05033  ],
       [  33.0334   ,   32.916153 ,   31.798866 , ...,   50.11741  ,
          47.708984 ,   43.5809   ],
       ...,
       [   5.1572466,    6.7922773,    2.9218807, ...,   -6.220009 ,
          -5.8999786,   -3.3235703],
       [   8.37487  ,   10.623951 ,    8.351871 , ...,   -6.296741 ,
          -6.197798 ,   -3.330977 ],
       [   9.6939335,    5.9644666,    1.5193555, ...,   -5.871545 ,
          -5.287879 ,   -2.399074 ]], dtype=float32), array([[-560.3812    , -560.3812    , -560.3812    , ..., -433.66232   ,
        -430.94424   , -433.11075   ],
       [   0.        ,    0.        ,    0.        , ...,  113.59108   ,
         113.47917   ,  115.02912   ],
       [   0.        ,    0.        ,    0.        , ...,   41.705498  ,
          39.950764  ,   41.72312   ]

C:\Users\Macey Cohn\AppData\Local\Temp\ipykernel_20648\1989804496.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mfccArray = np.array([elem for elem in enumerate(mfccs)])


[array([[-407.85757  , -387.25946  , -377.90118  , ..., -441.32175  ,
        -444.9362   , -448.96732  ],
       [  80.291916 ,   94.30681  ,   96.39616  , ...,   59.661114 ,
          55.342346 ,   50.05033  ],
       [  33.0334   ,   32.916153 ,   31.798866 , ...,   50.11741  ,
          47.708984 ,   43.5809   ],
       ...,
       [   5.1572466,    6.7922773,    2.9218807, ...,   -6.220009 ,
          -5.8999786,   -3.3235703],
       [   8.37487  ,   10.623951 ,    8.351871 , ...,   -6.296741 ,
          -6.197798 ,   -3.330977 ],
       [   9.6939335,    5.9644666,    1.5193555, ...,   -5.871545 ,
          -5.287879 ,   -2.399074 ]], dtype=float32), array([[-560.3812    , -560.3812    , -560.3812    , ..., -433.66232   ,
        -430.94424   , -433.11075   ],
       [   0.        ,    0.        ,    0.        , ...,  113.59108   ,
         113.47917   ,  115.02912   ],
       [   0.        ,    0.        ,    0.        , ...,   41.705498  ,
          39.950764  ,   41.72312   ]

ValueError: setting an array element with a sequence.